In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os, random, time, MirandaAttributes, sys
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import backend as K
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split, KFold

In [2]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [3]:
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

#Miranda dataset
train_dataset_url = "https://raw.githubusercontent.com/intentodemusico/StegianV2/master/DatasetMiranda/features_train_70000.csv"
test_dataset_url = "https://raw.githubusercontent.com/intentodemusico/StegianV2/master/DatasetMiranda/features_test_70000.csv"

TensorFlow version: 2.3.0
Eager execution: True


In [4]:
#%% Importing the dataset
MirandaLabels=['Kurtosis', 'Skewness', 'Std', 'Range', 'Median', 'Geometric_Mean', 'Mobility', 'Complexity']
trainDataset = pd.read_csv(train_dataset_url)
X_train = trainDataset.iloc[:, :-1].values
Y_train = trainDataset.iloc[:, -1].values

testDataset = pd.read_csv(test_dataset_url)
X_test = testDataset.iloc[:, :-1].values
Y_test = testDataset.iloc[:, -1].values

#%% Feature Scaling
from sklearn.model_selection import train_test_split
X_train, X_test, Y_Train, Y_Test = train_test_split(x, y, test_size=0.20, random_state=42)
from sklearn.preprocessing import StandardScaler
sc.fit_transform(


In [5]:
sc = StandardScaler()
X_Train = sc.fit_transform(X_train)
X_Test = sc.transform(X_test)

In [6]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(8, activation=tf.nn.relu, input_shape=(8,)),  # input shape required
  tf.keras.layers.Dense(4, activation=tf.nn.relu),
  tf.keras.layers.Dense(2, activation=tf.nn.relu),
  tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

model.fit(X_Train, Y_train, batch_size = 64, epochs = 250)

Epoch 1/250
875/875 [==============================] - 1s 1ms/step - loss: 0.3168 - accuracy: 0.8696
Epoch 2/250
875/875 [==============================] - 1s 1ms/step - loss: 0.1875 - accuracy: 0.9100
Epoch 3/250
875/875 [==============================] - 1s 1ms/step - loss: 0.1823 - accuracy: 0.9117
Epoch 4/250
875/875 [==============================] - 1s 1ms/step - loss: 0.1796 - accuracy: 0.9125
Epoch 5/250
875/875 [==============================] - 1s 1ms/step - loss: 0.1772 - accuracy: 0.9129
Epoch 6/250
875/875 [==============================] - 1s 1ms/step - loss: 0.1744 - accuracy: 0.9132
Epoch 7/250
875/875 [==============================] - 1s 1ms/step - loss: 0.1715 - accuracy: 0.9143
Epoch 8/250
875/875 [==============================] - 1s 1ms/step - loss: 0.1689 - accuracy: 0.9143
Epoch 9/250
875/875 [==============================] - 1s 1ms/step - loss: 0.1665 - accuracy: 0.9146
Epoch 10/250
875/875 [==============================] - 1s 1ms/step - loss: 0.1645 - accura

In [7]:
print(model)
model.evaluate(X_Train, Y_train)
print(model.metrics_names)
#%% Predicting the Test set results

1750/1750 [==============================] - 1s 829us/step - loss: 0.1399 - accuracy: 0.9336
['loss', 'accuracy']


In [8]:
from sklearn import metrics

In [9]:
y_pred = (model.predict(X_Test) > 0.5)
#%% Making the Confusion Matrix
cm = metrics.confusion_matrix(Y_test, y_pred)
print("Confusion matrix\n",cm)

sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity :', sensitivity )

specificity = cm[1,1]/(cm[1,0]+cm[1,1])
print('Specificity :', specificity)
#%% 
print("Precision",metrics.precision_score(Y_test, y_pred, average=None))
print("Accuracy",metrics.accuracy_score(Y_test, y_pred))
print("F1",metrics.f1_score(Y_test, y_pred, average=None))

Confusion matrix
 [[6665  334]
 [ 664 6336]]
Sensitivity : 0.9522788969852836
Specificity : 0.9051428571428571
Precision [0.90940101 0.94992504]
Accuracy 0.9287091935138224
F1 [0.93034618 0.92699342]


In [10]:
import numpy as np


In [11]:
fallas=np.array(labels,dtype='<U256')
y_pred=y_pred.reshape(len(y_pred))
print(np.shape(y_pred))
print(np.shape(Y_Test))
for i in range(len(y_pred)):
    if(y_pred[i]!=Y_Test[i]>0.5):
        fallas=np.vstack((fallas,testDataset.iloc[i].to_numpy()))

y_pred = (model.predict(X_Train) > 0.5).reshape(len(X_Train))
for i in range(len(y_pred)):
    if(y_pred[i]!=Y_Train[i]>0.5):
        fallas=np.vstack((fallas,trainDataset.iloc[i].to_numpy()))

asd="fallasEval.csv"
print("Fallas",len(fallas))
np.savetxt(asd,fallas,delimiter=",", fmt="%s")

NameError: name 'labels' is not defined